# Оценка результата A/B теста

**Цель проекта –** провести оценку результатов A/B-теста. 

*В качестве входных данных используется датасет с действиями пользователей и несколько вспомогательных датасетов.*

**Ход проекта:**

1. Обзор данных;
2. Предобработка данных;
3. Оценка корректности проведения теста;
4. Исследовательский анализ данных;
5. Анализ результатов теста;
6. Проверка статистических гипотез.

In [1]:
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from scipy import stats as st

In [2]:
pio.templates.default = 'seaborn'

In [3]:
events           = pd.read_csv('final_ab_events.csv')
marketing_events = pd.read_csv('final_ab_marketing_events.csv')
new_users        = pd.read_csv('final_ab_new_users.csv')
participants     = pd.read_csv('final_ab_participants.csv')

## Обзор данных

### В таблице events

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [5]:
events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


In [6]:
events.duplicated().sum()

0

**Согласно документации к данным:**

*events – действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.*

- `user_id` – идентификатор пользователя;  
- `event_dt` – дата и время покупки; 
- `event_name` – тип события;  
- `details` – дополнительные данные о событии. Например, для покупок `purchase`, в этом поле хранится стоимость покупки в долларах.

### В таблице marketing_events

In [7]:
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [8]:
marketing_events.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


In [9]:
marketing_events.duplicated().sum()

0

**Согласно документации к данным:**

*marketing_events – календарь маркетинговых событий на 2020 год.*

- `name` – название маркетингового события;  
- `regions` – регионы, в которых будет проводиться рекламная кампания; 
- `start_dt` – дата начала кампании;  
- `finish_dt` – дата завершения кампании.

### В таблице new_users

In [10]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [11]:
new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [12]:
new_users.duplicated().sum()

0

**Согласно документации к данным:**

*new_users – пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.*

- `user_id` – идентификатор пользователя;  
- `first_date` – дата регистрации; 
- `region` – регион пользователя;  
- `device` – устройство, с которого происходила регистрация.

### В таблице participants

In [13]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [14]:
participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [15]:
participants.duplicated().sum()

0

**Согласно документации к данным:**

*participants – таблица участников тестов.*

- `user_id` – идентификатор пользователя;  
- `group` – название теста; 
- `ab_test` – группа пользователя.

### Вывод

- В таблице events есть пропуски в столбце `details`;
- Даты в таблицах представлены неправильным типом данных.

## Предобработка данных

**Что бы предобработать данные, выполним следующие действия:**

- Приведем даты к типу данных datetime;
- Обработаем пропуски.

### Приведение типов

In [16]:
events['event_dt'] = pd.to_datetime(events['event_dt'], format='%Y-%m-%d %H:%M:%S')

In [17]:
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'], format='%Y-%m-%d')

In [18]:
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'], format='%Y-%m-%d')

In [19]:
new_users['first_date'] = pd.to_datetime(new_users['first_date'], format='%Y-%m-%d')

### Обработка пропусков

In [20]:
events[events['details'].isna()]['event_name'].value_counts()

login           189552
product_page    125563
product_cart     62462
Name: event_name, dtype: int64

*Пропуски получились из-за того что для вышеобозначенных событий не требовалось числового уточнения, можно заполнить нулем.*

In [21]:
events['details'] = events['details'].fillna(0)

In [22]:
events.isna().sum()

user_id       0
event_dt      0
event_name    0
details       0
dtype: int64

### Вывод

- Даты были приведены к формату даты;
- Пропуски заполнены нулем.

## Оценка корректности проведения теста

**Чтобы оценить корректность проведения теста, проверим его на соответствие техническому заданию:**

- Название теста: `recommender_system_test`;
- Группы: А — контрольная, B — новая платёжная воронка;
- Дата запуска: 2020-12-07;
- Дата остановки набора новых пользователей: 2020-12-21;
- Дата остановки: 2021-01-04;
- Аудитория: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемое количество участников теста: 6000;
- Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
  - Конверсии в просмотр карточек товаров — событие `product_page`; 
  - Просмотры корзины — `product_cart`;
  - Покупки — `purchase`.

### Проверка пересечения тестовой аудитории с конкурирующим тестом

**Посмотрим как разделились пользователи:**

In [23]:
participants.groupby('ab_test').agg({'user_id': 'nunique', 'group': 'nunique'})

,user_id,group
ab_test,,
interface_eu_test,11567,2
recommender_system_test,6701,2


*Название теста совпадает, групп тоже две.*

In [24]:
recommender_system_test = participants[participants['ab_test'] == 'recommender_system_test']

In [25]:
interface_eu_test = participants[participants['ab_test'] == 'interface_eu_test']

In [26]:
cross_users = recommender_system_test[recommender_system_test['user_id'].isin(interface_eu_test['user_id'])]
cross_users

,user_id,group,ab_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
10,B3A2485649E4A012,A,recommender_system_test
25,EAFB9027A27D510C,B,recommender_system_test
29,5D5E6EE92AF6E9E0,B,recommender_system_test
...,...,...,...
6323,C2DC7B6881CE6E30,B,recommender_system_test
6338,EA29547AB3C0CB9C,B,recommender_system_test
6341,9A44E27079666291,B,recommender_system_test
6343,FA86D8DFAD3ADADE,A,recommender_system_test


In [27]:
len(cross_users) / len(recommender_system_test)

0.23906879570213402

*Имеется 1602 (24%) пересечения с конкурирующим тестом.*

**Чтобы не было искажений в дальнейшем, удалим эти пересечения:**

In [28]:
recommender_system_test = recommender_system_test.drop(cross_users['user_id'].index)

**Теперь посмотрим не пересекаются ли группы между собой:**

In [29]:
wrong_users = (recommender_system_test.groupby('user_id')
               .agg({'group' : 'nunique'})
               .query('group > 1')
               .reset_index())['user_id']
wrong_users = recommender_system_test[recommender_system_test['user_id'].isin(wrong_users)]

In [30]:
wrong_users['user_id'].nunique()

0

*Межгруппового пересечения нет.*

**Посмотрим на разделение на тестовую и контрольную:**

In [31]:
recommender_system_test['group'].value_counts()

A    2903
B    2196
Name: group, dtype: int64

In [32]:
(len(recommender_system_test[recommender_system_test['group'] == 'A']) / 
len(recommender_system_test[recommender_system_test['group'] == 'B'])
) - 1

0.3219489981785064

*Группы разделились неравномерно.*

### Проверка времени запуска и остановки набора новых пользователей

**Объединим таблицы `recommender_system_test` и `new_users`:**

In [33]:
df = recommender_system_test.merge(new_users[new_users['region'] == 'EU'], on='user_id')

In [34]:
df['first_date'].min()

Timestamp('2020-12-07 00:00:00')

In [35]:
df['first_date'].max()

Timestamp('2020-12-21 00:00:00')

*Время запуска и остановки набора новых пользователей соответствуют техническому заданию.*

### Проверка совпадения теста и маркетинговых событий

In [36]:
marketing_events[marketing_events['start_dt'].between('2020-12-07', '2021-01-04')]

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
10,CIS New Year Gift Lottery,CIS,2020-12-30,2021-01-07


*Во время проведения теста в регионе EU проходило маркетинговое событие `Christmas&New Year Promo`.*

**Оценим его влияние на тест:**

In [37]:
# Выделим отдельный столбец date
events['event_date'] = pd.to_datetime(events['event_dt'].dt.date)

In [38]:
events_count = events.groupby('event_date')['event_name'].count().reset_index()

In [39]:
fig = px.bar(
    events_count, 
    x='event_date', 
    y='event_name',
    labels={'event_date': 'Дата', 'event_name': 'Количество'},
)

fig.update_xaxes(tickangle=45, tickvals=events_count['event_date'], showgrid=False)
fig.update_layout(title_text='Количество событий по времени', height=700)
fig.show()

*Количество событий после 21 декабря 2020 года уменьшается, маркетинговое событие не оказывает влияние на проведение теста, так же видно что последнее событие совершено 30 декабря 2020, значит время остановки теста не соответствует техническому заданию и не все когорты дожили до 14 дня.*

**Оставим только тех пользователей, для которых лайфтайм соответствует 14 дням:**

In [40]:
df = df.merge(events, on='user_id', how='left')

In [41]:
df['observation_date'] = df['first_date'] + pd.Timedelta(days = 14)

In [42]:
df = df[df['event_date'] <= df['observation_date']]

### Проверка аудитории теста

**Проверим, какая часть новых пользователей попала в тест, должно быть 15% из EU:**

In [43]:
df['user_id'].nunique() / new_users[new_users['region'] == 'EU']['user_id'].nunique()

0.0560622433542252

*5.6 %, что не соотвествует техническому заданию.*

### Проверка количества уникальных пользователей

In [44]:
df['user_id'].nunique()

2594

*Должно быть 6000, имеется 2594, это получилось из-за того, что были удалены пользователи, которые участвовали одновременно в конкурирующем тесте и те, у кого лайфтайм не соответствует 14 дням.*

### Вывод

- Название теста и количество групп соответствует техническому заданию;
- Время начала и остановки набора пользователей, а так же время остановки теста соответствует техническому заданию;
- Аудитория для теста набрана неправильно, вместо ожидаемых 15% из EU было всего 5.6%;
- Распределение пользователей между тестами осуществленно неверно, 23% попали в конкурирующий тест;
- Разделение на контрольную и тестовую так же выполнено неверно, преобладание одной над другой;
- Количество пользователей для теста меньше чем предполагалось, из-за пересечения с конкурирующим тестом;
- Во время теста проходило маркетинговое событие, но оно не оказывало влияние на тест.

**Из всего этого можно сделать вывод, что система по разделению пользоватлей работает некорректно и нуждается в доработке.**

## Исследовательский анализ данных

**Чтобы провести исследовательский анализ данных, выполним следующие действия:**

- Проверим распределение количества событий на пользователя по группам;
- Проверим распределение числа событий в группах по дням;
- Выясним как меняется конверсия в воронке в группах на разных этапах;
- Выясним какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию.

### Распределение количества событий на пользователя по группам

In [45]:
event_count = df.groupby('user_id', as_index=False).agg({'event_name':'count', 'group': 'first'})

In [46]:
fig = px.histogram(event_count, 
                   x='event_name',
                   color='group',
                   barmode='overlay',
                   opacity=0.7,
                   labels={'event_name': 'Количество событий'}
)
fig.update_xaxes(showgrid=False)
fig.update_layout(
    title_text='Распределение количества событий на пользователя по группам', 
    xaxis_title_text='Количество событий', 
    yaxis_title_text='Количество пользователей',
    height=700
)
fig.show()

In [47]:
event_count.groupby('group', as_index=False).agg({'event_name': ['count', 'min', 'max', 'mean']})

group event_name                  
             count min max      mean
0     A       1939   1  24  6.956679
1     B        655   1  24  5.493130

*Количество событий на пользователя отличается по группам. В среднем на пользователя в контрольной группе приходится 7 событий, а в тестовой 5.5.*

### Распределение числа событий в группах по дням

In [48]:
fig = px.histogram(
    df, 
    x='event_date', 
    y='event_name',
    color='group',                                      
    barmode='group',
    histfunc='count',
    labels={'event_date': 'Дата', 
            'group': 'Группа'
    },
)
fig.update_xaxes(tickangle=45, tickvals=events_count['event_date'], showgrid=False)
fig.update_layout(
    title_text='Распределение числа событий в группах по дням', 
    xaxis_title_text='Дата', 
    yaxis_title_text='Количество событий',
    height=700
)
fig.show()

- Только 7 декабря количество событий в тестовой группе было больше, чем в контрольной;
- С 14 декабря наблюдается резкий рост количества событий в контрольной группе, пик приходится на 21 декабря, после количество падает;
- 30 декабря события в обеих группах практически отсутствуют.

### Изменение конверсии в воронке в группах на разных этапах

**Сначала определим воронку:**

In [49]:
events_by_group = df.pivot_table(values='user_id', index='group', columns='event_name', aggfunc='nunique').reset_index()

In [50]:
events_by_group = (events_by_group
                    .merge(event_count
                            .groupby('group', as_index=False)
                            .agg({'event_name': 'count'})
                            .rename(columns={'event_name': 'users_count'}), on='group'
                          )
)

In [51]:
events_by_group = events_by_group[['group', 'users_count', 'login', 'product_page', 'product_cart', 'purchase']]

In [52]:
events_by_group

,group,users_count,login,product_page,product_cart,purchase
0,A,1939,1939,1265,589,613
1,B,655,654,367,184,191


In [53]:
events_funnel = events_by_group.set_index('group').T

**Теперь с помощью воронки посмотрим на конверсию:**

In [54]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A',
    y = events_funnel.index,
    x = events_funnel['A'],
    hoverinfo = "y",
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'B',
    y = events_funnel.index,
    x = events_funnel['B'],
    hoverinfo = "y",
    textinfo = "value+percent previous"))

fig.update_layout(title_text='Воронка в группах на разных этапах',
                  height=700
)
fig.show()

- Ни на одном из этапов не видно увеличения конверсии на 10%;
- Больше всего пользователей теряется на моменте просмотра корзины;
- Покупок совершается больше, чем просмотров корзины, скорее всего есть механизм быстрых заказов, минующий остальные этапы.

### Вывод

- Количество событий на пользователя отличается по группам;
- В контрольной группе событий в разы больше, чем в тестовой;
- Много пользователей, которые не совершили ни одного действия;
- Только 7 декабря количество событий в тестовой группе было больше, чем в контрольной;
- С 14 декабря наблюдается резкий рост количества событий в контрольной группе, пик приходится на 21 декабря, после количество падает;
- 30 декабря события в обеих группах практически отсутствуют;
- В тестовой группе изначально, только 32% пользователей совершили логин;
- Ни на одном из этапов не видно увеличения конверсии на 10%;
- Больше всего пользователей теряется на моменте просмотра корзины;
- Покупок совершается больше, чем просмотров корзины, скорее всего есть механизм быстрых заказов, минующий остальные этапы.

**Перед проведением статистических тестов нужно учесть, что разделение на группы было выполнено некорректно и что не все пользователи проявили активность в момент проведения теста.**

## Проверка статистических гипотез

**Проверим гипотезу о равенстве долей для событий: `product_page`, `product_cart`, `purchase`.**  
**Сформулируем нулевую гипотезу:** значимой разницы между долями нет.  
**Сформулируем алтернативную гипотезу:** значимая разницы между долями есть.

In [55]:
def check_equality_of_share(count1, count2, all_count1, all_count2):
    """
    Функция для проведения Z-теста
    
    count1 - размер первой выборки
    count2 - размер второй выборки
    all_count1 - размер первой генеральной совокупности
    all_count2 - размер второй генеральной совокупности
    """
    
    share_of_all = np.array([count1, count2])
    all_count = np.array([all_count1, all_count2])
    
    # пропорция в первой группе:
    p1 = share_of_all[0] / all_count[0]

    # пропорция во второй группе:
    p2 = share_of_all[1] / all_count[1]
    
    # пропорция в комбинированном датасете:
    p_combined = (share_of_all[0] + share_of_all[1]) / (all_count[0] + all_count[1])
    
    # статистика в ст.отклонениях стандартного нормального распределения
    z_value = (p1 - p2) / math.sqrt(p_combined * (1 - p_combined) * (1/all_count[0] + 1/all_count[1]))
    
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1) 
    
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    return p_value[0]

In [56]:
def check_group(data, group1, group2, all_count1, all_count2, alpha=.05):
    """
    Функция для сравнения долей двух выборок

    data - датафрейм
    group1 - номер группы 1
    group2 - номер группы 2
    all_count1 - размер первой генеральной совокупности
    all_count2 - размер второй генеральной совокупности
    alpha - критический уровень статистической значимости
    """
    alpha = alpha
    a = data[data['group'] == group1]
    b = data[data['group'] == group2]
        
    for x in range(1, len(data.columns)):
        print(f'Для групп {group1} и {group2} проверка {data.columns[x]}:')
        p_value = check_equality_of_share(a.iloc[:, x], 
                                          b.iloc[:, x], 
                                          all_count1, 
                                          all_count2
                                         )
        print('p-значение: ', p_value)
        if p_value < alpha:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')
        print('\n')

In [57]:
users_count_a = events_by_group[events_by_group['group'] == 'A']['users_count']

In [58]:
users_count_b = events_by_group[events_by_group['group'] == 'B']['users_count']

**Производится множественное тестирование, с поправкой Бонферрони критический уровень статистической значимости будет 0.05/3 = 0.017**

In [59]:
check_group(events_by_group.drop(['users_count', 'login'], axis=1), 'A', 'B', users_count_a, users_count_b, 0.017)

Для групп A и B проверка product_page:
p-значение:  2.457788807452843e-05
Отвергаем нулевую гипотезу: между долями есть значимая разница


Для групп A и B проверка product_cart:
p-значение:  0.26899325584838074
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


Для групп A и B проверка purchase:
p-значение:  0.24035836270660704
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными




### Вывод

Нулевая гипотеза отвергнута только для страницы продукта.

## Общий вывод

**По результатам воронки продаж, а так же результатам A/B-тестирования, можно сделать вывод:**

**Тест был выполнен некорректно, он не соответствует ни правилам проведения A/B-тестирования, ни техническому заданию, были выявлены следующие недостатки:**

- Аудитория для теста набрана неправильно, вместо ожидаемых 15% из EU было всего 5.6%;
- Распределение пользователей между тестами осуществленно неверно, 23% попали в конкурирующий тест;
- Разделение на контрольную и тестовую так же выполнено неверно, преобладание одной над другой;
- Количество пользователей для теста меньше чем предполагалось, из-за пересечения с конкурирующим тестом;
- Много пользователей, которые не совершили ни одного действия.
- Последнее событие было 30 декабря 2020, хотя тест предполагался до 4 января 2021;
- Ни на одном из этапов не видно увеличения конверсии на 10%.

**Рекомендации:** учесть недочеты и доработать механизм разделения на группы.